In [1]:
# mysql connection
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [6]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("tiem_tfidf") \
    .master("spark://node01:10077") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/19 02:13:07 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [7]:
# import jieba and set stop word set
import jieba
import jieba.analyse

stop_words_rdd = sc.textFile("hdfs:///user/spark_temp/stopwords.dat")
stop_words_set = set(stop_words_rdd.collect())

In [8]:
# define map functions 
from datetime import datetime

date_string = datetime.today().strftime('%Y-%m-%d')
# cut item name and description
def cut_name_and_desc(item):
    id, name, desc = item
    name_cut = set(jieba.cut(name))
    name_cut_pure = set(jieba.cut(name)) - stop_words_set
    desc_cut = set(jieba.cut(desc))
    desc_cut_pure = set(jieba.cut(desc)) - stop_words_set
    return (id,name_cut,name_cut_pure,desc_cut,desc_cut_pure)

# map item's cut list to word count
def to_count(item):
    id,name_cut,name_cut_pure,desc_cut,desc_cut_pure = item
    for i in name_cut_pure:
        yield ((id, i),1)
    for i in desc_cut_pure:
        yield ((id,i),1)
        
# trnasfer (id, key), count to id,key,count, date
def split_key_set_date(item):
    key1,count = item
    id,key = key1
    date = date_string
    return id,key,count,date

In [10]:
# get all item data from db, set result to an RDD
cur = mydb.cursor()
cur.execute("SELECT id,name,description from items order by RAND() ")
result = cur.fetchall()
all_items = sc.parallelize(result)
print(all_items.count())

433


In [12]:
# cut item's name and description
cut_items = all_items.map(cut_name_and_desc)
print(all_items.count())

433


In [13]:
# do word count
item_word_count = cut_items.flatMap(to_count)\
                    .reduceByKey(lambda a,b:a+b)\
                    .map(split_key_set_date)
print(item_word_count.count())

4638


In [14]:
# create a table for wordcount
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("item_id", IntegerType(), True),
    StructField("key_word", StringType(), True),
    StructField("word_count", IntegerType(), True),
    StructField("date", StringType(),True)
])
item_word_count = spark.createDataFrame(item_word_count, schema)
item_word_count.createOrReplaceTempView("item_word_count")
item_word_count.show()

+-------+--------+----------+----------+
|item_id|key_word|word_count|      date|
+-------+--------+----------+----------+
|    218|      牛|         2|2023-04-19|
|    218|      药|         1|2023-04-19|
|    218|    用于|         1|2023-04-19|
|     87|    佳品|         1|2023-04-19|
|     87|    水分|         1|2023-04-19|
|     87|    传统|         1|2023-04-19|
|     87|    优质|         1|2023-04-19|
|     87|    精选|         1|2023-04-19|
|     87|    烹饪|         1|2023-04-19|
|     16|      斤|         1|2023-04-19|
|     16|    健康|         1|2023-04-19|
|     16|    放养|         1|2023-04-19|
|     16|    极佳|         1|2023-04-19|
|     68|  酸酸乳|         2|2023-04-19|
|     68|    原味|         2|2023-04-19|
|     68|    蒙牛|         2|2023-04-19|
|     68|        |         1|2023-04-19|
|     68|    适中|         1|2023-04-19|
|     68|    口感|         1|2023-04-19|
|     68|    助手|         1|2023-04-19|
+-------+--------+----------+----------+
only showing top 20 rows



In [22]:
# compute IDF
item_word_idf = spark.sql('''
select
    key_word,
    count(distinct item_id) as item_num_has_word,
    max(a.item_num) as item_num_all,
    log10(max(a.item_num)/ count(distinct item_id)) as idf
from
    item_word_count,(
        select
            count(distinct item_id) as item_num
        from
            item_word_count
    ) as a
group by
    key_word
''')
item_word_idf.show()

+--------+-----------------+------------+------------------+
|key_word|item_num_has_word|item_num_all|               idf|
+--------+-----------------+------------+------------------+
|    肉质|              101|         433|0.6321665225707229|
|    屠宰|               17|         433|1.4060389749750914|
|    产后|                1|         433|2.6364878963533656|
|    一号|               22|         433|1.2940652155311592|
|    体重|                4|         433| 2.034427905025403|
|    助手|                1|         433|2.6364878963533656|
|      猪|                4|         433| 2.034427905025403|
|    品质|               11|         433|1.5950952111951404|
|      约|                7|         433|1.7913898563391086|
|    放养|                3|         433| 2.159366641633703|
|    效益|                3|         433| 2.159366641633703|
|    塔尔|                2|         433|2.3354579006893843|
|    独特|                4|         433| 2.034427905025403|
|    消毒|                2|         433|2.3354579

In [5]:
# close spark session
spark.stop()